In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

In [10]:
def open_file():
    with open('ddata_min.pickle', 'rb') as f:
        f_data = pickle.load(f)
    return f_data

In [11]:
def coin_select():
    f_data = open_file()
    coin_name = str(input('Enter the coin name: '))
    s_coin_data = BTC_data = f_data[f_data.permno == coin_name].copy()
    s_coin_data = s_coin_data.reset_index()
    return s_coin_data

In [18]:
def RSI(period=5):
    s_coin_data = coin_select()
    period = period
    U = np.where(s_coin_data.prc.diff(1) > 0, s_coin_data.prc.diff(1), 0)
    D = np.where(s_coin_data.prc.diff(1) < 0, s_coin_data.prc.diff(1)*(-1), 0)
    AU = pd.DataFrame(U, index=s_coin_data.index).rolling(window=period).mean()
    AD = pd.DataFrame(D, index=s_coin_data.index).rolling(window=period).mean()
    RSI = AU / (AD+AU) * 100
    s_coin_data['RSI'] = RSI
    s_coin_data['Signal'] = None
    return s_coin_data

In [26]:
def trading_func(initial_asset=100, sell_fee=0, buy_fee=0, period=5):
    s_coin_data = RSI(period=period)
    
    bid_price = 0
    total_profit = 0
    total_loss = 0
    win_count = 0
    lose_count = 0
    sell_signal = False
    initial_asset = initial_asset
    sell_fee = sell_fee
    buy_fee = buy_fee
    
    for i, row in s_coin_data.iterrows():
        date, price, rsi, Signal = row['date'], row['prc'], row['RSI'], row['Signal']
    
        
        if sell_signal:
            #수익률 계산
            profit_pct = (price - bid_price) / bid_price * 100 - sell_fee
            s_coin_data.at[i, 'Profit'] = profit_pct * initial_asset
            s_coin_data.Signal[i] = 'Sell'

            #초기화
            bid_price = 0
            sell_signal = False

            #수익
            if profit_pct > 0:
                win_count += 1
                total_profit += profit_pct * initial_asset

            #손실
            else:
                lose_count += 1
                total_loss += profit_pct * initial_asset

            initial_asset += profit_pct * initial_asset
#             print('Date: {} | Price: {:7.2f} | Profit: {:5.2f}'.format(date, price, profit))
            print(initial_asset, profit_pct)
        #Buy signal
        elif not bid_price and rsi <= 30:
            s_coin_data.Signal[i] = 'Buy'
            initial_asset = initial_asset * (1 - buy_fee)
            bid_price = s_coin_data.prc[i]
#             print('Date: {} | Buy Price: {}'.format(date, price))

        #Sell signal
        elif bid_price and rsi >= 70:
            sell_signal = True
        
    return print('Final Asset: {:10.4f} | Winning Trade: {:7} | Losing Trade: {:7}'.
                format(initial_asset, win_count, lose_count))

In [21]:
trading_func(initial_asset=100, sell_fee=0.0004, buy_fee=0.0004)

Enter the coin name:  BTC


<ipython-input-20-23e4586146d5>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_coin_data.Signal[i] = 'Buy'
<ipython-input-20-23e4586146d5>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_coin_data.Signal[i] = 'Sell'


Final Asset:     0.0000 | Winning Trade:   30243 | Losing Trade:   17543


In [28]:
if __name__ == "__main__":
    trading_func(initial_asset=100, sell_fee=0.0004, buy_fee=0.0004)

Enter the coin name:  BTC


<ipython-input-26-6a2a08905433>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_coin_data.Signal[i] = 'Buy'
<ipython-input-26-6a2a08905433>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_coin_data.Signal[i] = 'Sell'


109.543074186738 0.0958690895031812
108.65109387435209 -0.007745834138799712
110.37083211929601 0.01623457418874386
111.74185851276498 0.012827130098969833
111.62011482792931 -0.0006897842363234382
106.93094141785888 -0.041626761671667324
97.54516530096006 -0.08740914756171365
110.11998306916092 0.12936451897685292
110.48894063396722 0.003752005901644687
113.06805278955237 0.023752218636277384
115.1951207604663 0.019219968896587118
109.01934723764325 -0.05323270893220765
114.65633792558644 0.05212718402249057
115.12712848117636 0.004507904613432438
115.20117016607823 0.0010435471996592864
124.31404420782087 0.07953582020410863
121.0467617846142 -0.025892845524997706
122.5455995218462 0.012787418427078067
86.03425244272867 -0.29765997105958036
90.48500567066338 0.05215320161681095
65.61607868996994 -0.27455012974042536
71.85216472589376 0.09547718376103673
71.87824572221642 0.0007632866726024795
65.84383879241781 -0.0835866094766799
69.46391895741786 0.05540194676009045
67.5891810495337

KeyboardInterrupt: 